# Market Intelligence LLM Application

A comprehensive market intelligence application that combines **Tavily AI** for real-time data retrieval and **OpenAI GPT-4o-mini** for intelligent analysis and insight generation.

## 🚀 Features

- **Real-time Data Retrieval**: Fetch data from social media, news, and reports using Tavily AI
- **Three Analysis Types**: Market trends, customer sentiment, and competitive analysis
- **Batch Processing**: Analyze multiple queries simultaneously
- **Interactive Interface**: User-friendly query system
- **Comprehensive Dashboards**: Visual representation of analysis results
- **Export Functionality**: Save results in multiple formats (TXT, CSV)
- **Error Handling**: Robust error management and logging

## Prerequisites

Before running this notebook, ensure you have:
- Python 3.8 or higher
- API keys for Tavily AI and OpenAI
- All required packages installed (see requirements.txt)

## Setup Instructions

1. Install required packages from requirements.txt
2. Set up your API keys as environment variables:
   - `TAVILY_API_KEY`: Your Tavily AI API key
   - `OPENAI_API_KEY`: Your OpenAI API key
3. Run the cells sequentially

## 📦 Package Installation

Run the following cell to install all required packages:

In [ ]:
# Install required packages
!pip install langchain>=0.1.0 langchain-openai>=0.1.0 langchain-community>=0.0.20 tavily-python>=0.3.0 python-dotenv>=1.0.0 pandas>=1.5.0 matplotlib>=3.6.0 seaborn>=0.12.0 jupyter>=1.0.0

## 📚 Import Required Libraries

In [ ]:
# Import required libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# LangChain and AI imports
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from tavily import TavilyClient

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")

## 🔐 Environment Setup

Load environment variables and configure API keys:

In [ ]:
# Load environment variables
load_dotenv()

# Get API keys from environment variables
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Check if API keys are loaded
if not TAVILY_API_KEY:
    print("⚠️  TAVILY_API_KEY not found in environment variables")
    print("Please set your Tavily API key:")
    TAVILY_API_KEY = input("Enter your Tavily API key: ")
    
if not OPENAI_API_KEY:
    print("⚠️  OPENAI_API_KEY not found in environment variables")
    print("Please set your OpenAI API key:")
    OPENAI_API_KEY = input("Enter your OpenAI API key: ")

print("✅ API keys configured successfully!")

## 🏗️ Market Intelligence Application Class

Main application class that handles data retrieval, analysis, and insights generation:

In [ ]:
class MarketIntelligenceApp:
    def __init__(self):
        """Initialize the Market Intelligence Application."""
        self.tavily_client = TavilyClient(api_key=TAVILY_API_KEY)
        self.llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.3,
            openai_api_key=OPENAI_API_KEY
        )
        self.analysis_history = []
        
    def retrieve_data(self, query: str, max_results: int = 10) -> Dict[str, Any]:
        """Retrieve real-time data using Tavily AI."""
        try:
            response = self.tavily_client.search(
                query=query,
                search_depth="advanced",
                max_results=max_results
            )
            return response
        except Exception as e:
            print(f"Error retrieving data: {e}")
            return {"results": [], "error": str(e)}
    
    def analyze_with_llm(self, data: Dict[str, Any], query: str, analysis_type: str) -> str:
        """Analyze data using OpenAI GPT-4o-mini."""
        
        # Define analysis prompts based on type
        prompts = {
            "trends": """
            You are a market research analyst. Based on the following data, provide a comprehensive analysis of market trends.
            
            Focus on:
            1. Key market trends identified
            2. Growth opportunities
            3. Potential challenges
            4. Actionable recommendations for businesses
            
            Data: {data}
            Query: {query}
            
            Provide a detailed analysis:
            """,
            
            "sentiment": """
            You are a customer sentiment analyst. Based on the following data, analyze customer sentiment and feedback.
            
            Focus on:
            1. Overall sentiment distribution (positive, negative, neutral)
            2. Key themes in customer feedback
            3. Major concerns and pain points
            4. Opportunities for improvement
            
            Data: {data}
            Query: {query}
            
            Provide a detailed sentiment analysis:
            """,
            
            "competitive": """
            You are a competitive intelligence analyst. Based on the following data, provide a comprehensive competitive analysis.
            
            Focus on:
            1. Key competitors and their market positions
            2. Competitive advantages and strategies
            3. Market share insights
            4. Innovation trends
            
            Data: {data}
            Query: {query}
            
            Provide a detailed competitive analysis:
            """
        }
        
        prompt = prompts.get(analysis_type, prompts["trends"])
        
        try:
            # Format data for analysis
            formatted_data = self._format_data_for_analysis(data)
            
            # Create and execute the analysis chain
            messages = [
                SystemMessage(content="You are an expert market intelligence analyst."),
                HumanMessage(content=prompt.format(data=formatted_data, query=query))
            ]
            
            response = self.llm.invoke(messages)
            return response.content
            
        except Exception as e:
            return f"Error in analysis: {str(e)}"
    
    def _format_data_for_analysis(self, data: Dict[str, Any]) -> str:
        """Format retrieved data for LLM analysis."""
        if "results" not in data or not data["results"]:
            return "No data available for analysis."
        
        formatted_results = []
        for i, result in enumerate(data["results"][:10], 1):
            formatted_results.append(f"Source {i}: {result.get('title', 'N/A')} - {result.get('content', 'N/A')}")
        
        return "\n\n".join(formatted_results)
    
    def analyze(self, industry: str, query: str, analysis_type: str = "trends") -> Dict[str, Any]:
        """Main analysis method that combines data retrieval and LLM analysis."""
        timestamp = datetime.now().isoformat()
        
        # Enhance query with industry context
        enhanced_query = f"{industry} industry: {query}"
        
        print(f"🔍 Retrieving data for: {enhanced_query}")
        
        # Retrieve data
        data = self.retrieve_data(enhanced_query)
        
        if "error" in data:
            return {
                "timestamp": timestamp,
                "industry": industry,
                "query": query,
                "analysis_type": analysis_type,
                "error": data["error"]
            }
        
        print(f"📊 Analyzing {len(data.get('results', []))} data sources...")
        
        # Analyze with LLM
        insights = self.analyze_with_llm(data, query, analysis_type)
        
        # Create result object
        result = {
            "timestamp": timestamp,
            "industry": industry,
            "query": query,
            "analysis_type": analysis_type,
            "data_sources": len(data.get('results', [])),
            "insights": insights,
            "raw_data": data
        }
        
        # Add to history
        self.analysis_history.append({
            "timestamp": timestamp,
            "industry": industry,
            "analysis_type": analysis_type,
            "query": query,
            "sources": len(data.get('results', []))
        })
        
        print("✅ Analysis completed successfully!")
        
        return result
    
    def export_insights(self, result: Dict[str, Any], filename: str) -> None:
        """Export analysis results to a text file."""
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                f.write("Market Intelligence Report\n")
                f.write("=" * 60 + "\n")
                f.write(f"Generated: {result['timestamp']}\n")
                f.write(f"Industry: {result['industry']}\n")
                f.write(f"Query: {result['query']}\n")
                f.write(f"Analysis Type: {result['analysis_type']}\n")
                f.write(f"Data Sources: {result['data_sources']}\n")
                f.write("=" * 60 + "\n\n")
                f.write("INSIGHTS:\n")
                f.write(result['insights'])
            
            print(f"✅ Results exported to {filename}")
            
        except Exception as e:
            print(f"❌ Error exporting results: {e}")
    
    def get_analysis_history(self) -> pd.DataFrame:
        """Get analysis history as a DataFrame."""
        return pd.DataFrame(self.analysis_history)

print("✅ MarketIntelligenceApp class defined successfully!")

## 🔧 Helper Functions

Additional utility functions for batch analysis, visualization, and data export:

In [ ]:
def batch_analysis(app: MarketIntelligenceApp, queries: List[Dict[str, str]]) -> List[Dict[str, Any]]:
    """Perform batch analysis on multiple queries."""
    results = []
    
    print(f"🚀 Starting batch analysis of {len(queries)} queries...")
    
    for i, query_data in enumerate(queries, 1):
        print(f"\n📊 Processing query {i}/{len(queries)}...")
        
        result = app.analyze(
            industry=query_data.get('industry', 'general'),
            query=query_data.get('query', ''),
            analysis_type=query_data.get('analysis_type', 'trends')
        )
        
        results.append(result)
    
    print(f"\n✅ Batch analysis completed! Processed {len(results)} queries.")
    return results

def interactive_analysis(app: MarketIntelligenceApp) -> None:
    """Interactive analysis interface."""
    print("🎯 Interactive Market Intelligence Analysis")
    print("=" * 50)
    
    while True:
        print("\nAnalysis Types:")
        print("1. Market Trends (trends)")
        print("2. Customer Sentiment (sentiment)")
        print("3. Competitive Analysis (competitive)")
        print("4. Exit")
        
        choice = input("\nSelect analysis type (1-4): ").strip()
        
        if choice == '4':
            print("👋 Thank you for using Market Intelligence Analysis!")
            break
        
        analysis_types = {
            '1': 'trends',
            '2': 'sentiment',
            '3': 'competitive'
        }
        
        if choice not in analysis_types:
            print("❌ Invalid choice. Please try again.")
            continue
        
        industry = input("Enter industry: ").strip()
        query = input("Enter your query: ").strip()
        
        if not industry or not query:
            print("❌ Industry and query are required.")
            continue
        
        # Perform analysis
        result = app.analyze(
            industry=industry,
            query=query,
            analysis_type=analysis_types[choice]
        )
        
        # Display results
        print("\n" + "=" * 60)
        print("📊 ANALYSIS RESULTS")
        print("=" * 60)
        print(f"Industry: {result['industry']}")
        print(f"Query: {result['query']}")
        print(f"Analysis Type: {result['analysis_type']}")
        print(f"Data Sources: {result['data_sources']}")
        print("\n📈 INSIGHTS:")
        print(result['insights'])
        print("=" * 60)
        
        # Ask if user wants to save
        save = input("\nSave results to file? (y/n): ").strip().lower()
        if save == 'y':
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"analysis_{timestamp}.txt"
            app.export_insights(result, filename)

def create_analysis_dashboard(app: MarketIntelligenceApp) -> None:
    """Create a comprehensive analysis dashboard."""
    history_df = app.get_analysis_history()
    
    if history_df.empty:
        print("📊 No analysis history available. Run some analyses first!")
        return
    
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Market Intelligence Analysis Dashboard', fontsize=16, fontweight='bold')
    
    # Analysis type distribution
    analysis_counts = history_df['analysis_type'].value_counts()
    axes[0, 0].pie(analysis_counts.values, labels=analysis_counts.index, autopct='%1.1f%%')
    axes[0, 0].set_title('Analysis Type Distribution')
    
    # Industry distribution
    industry_counts = history_df['industry'].value_counts()
    axes[0, 1].bar(industry_counts.index, industry_counts.values)
    axes[0, 1].set_title('Industry Distribution')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Sources distribution
    axes[1, 0].hist(history_df['sources'], bins=10, alpha=0.7)
    axes[1, 0].set_title('Data Sources Distribution')
    axes[1, 0].set_xlabel('Number of Sources')
    axes[1, 0].set_ylabel('Frequency')
    
    # Timeline
    history_df['timestamp'] = pd.to_datetime(history_df['timestamp'])
    timeline_data = history_df.groupby(history_df['timestamp'].dt.date).size()
    axes[1, 1].plot(timeline_data.index, timeline_data.values, marker='o')
    axes[1, 1].set_title('Analysis Timeline')
    axes[1, 1].set_xlabel('Date')
    axes[1, 1].set_ylabel('Number of Analyses')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print("\n📊 DASHBOARD SUMMARY")
    print("=" * 40)
    print(f"Total Analyses: {len(history_df)}")
    print(f"Industries Analyzed: {history_df['industry'].nunique()}")
    print(f"Average Sources per Analysis: {history_df['sources'].mean():.1f}")
    print(f"Most Common Analysis Type: {history_df['analysis_type'].mode().iloc[0]}")
    print(f"Most Analyzed Industry: {history_df['industry'].mode().iloc[0]}")

def export_history_to_csv(app: MarketIntelligenceApp, filename: str) -> None:
    """Export analysis history to CSV file."""
    try:
        history_df = app.get_analysis_history()
        if history_df.empty:
            print("📊 No analysis history to export.")
            return
        
        history_df.to_csv(filename, index=False)
        print(f"✅ Analysis history exported to {filename}")
        
    except Exception as e:
        print(f"❌ Error exporting history: {e}")

print("✅ Helper functions defined successfully!")

## 🚀 Initialize Application

Create an instance of the Market Intelligence Application:

In [ ]:
# Initialize the Market Intelligence Application
app = MarketIntelligenceApp()

print("🎉 Market Intelligence Application initialized successfully!")
print("\n🎯 You can now:")
print("1. Run trend analysis: app.analyze(industry='tech', query='AI trends', analysis_type='trends')")
print("2. Analyze sentiment: app.analyze(industry='healthcare', query='patient satisfaction', analysis_type='sentiment')")
print("3. Competitive analysis: app.analyze(industry='fintech', query='market leaders', analysis_type='competitive')")
print("4. Run interactive analysis: interactive_analysis(app)")
print("5. Create dashboard: create_analysis_dashboard(app)")

## 📊 Example Usage: Market Trends Analysis

Analyze current trends in AI adoption and implementation:

In [ ]:
# Example 1: Market Trends Analysis
print("🔍 Example 1: Analyzing AI Adoption Trends")
print("=" * 50)

trends_result = app.analyze(
    industry="technology",
    query="What are the latest trends in AI adoption and implementation?",
    analysis_type="trends"
)

# Display results
if 'error' not in trends_result:
    print(f"\n📊 Analysis completed with {trends_result['data_sources']} sources")
    print(f"🕐 Generated: {trends_result['timestamp']}")
    print(f"🏢 Industry: {trends_result['industry']}")
    print(f"❓ Query: {trends_result['query']}")
    print(f"📈 Type: {trends_result['analysis_type']}")
    print("\n💡 Key Insights:")
    print(trends_result['insights'][:500] + "..." if len(trends_result['insights']) > 500 else trends_result['insights'])
else:
    print(f"❌ Error: {trends_result['error']}")

## 💭 Example Usage: Customer Sentiment Analysis

Analyze customer sentiment towards AI-powered diagnostic tools:

In [ ]:
# Example 2: Customer Sentiment Analysis
print("🔍 Example 2: Analyzing Customer Sentiment")
print("=" * 50)

sentiment_result = app.analyze(
    industry="healthcare",
    query="What is the customer sentiment towards AI-powered diagnostic tools?",
    analysis_type="sentiment"
)

# Display results
if 'error' not in sentiment_result:
    print(f"\n📊 Analysis completed with {sentiment_result['data_sources']} sources")
    print(f"🕐 Generated: {sentiment_result['timestamp']}")
    print(f"🏢 Industry: {sentiment_result['industry']}")
    print(f"❓ Query: {sentiment_result['query']}")
    print(f"📈 Type: {sentiment_result['analysis_type']}")
    print("\n💡 Key Insights:")
    print(sentiment_result['insights'][:500] + "..." if len(sentiment_result['insights']) > 500 else sentiment_result['insights'])
else:
    print(f"❌ Error: {sentiment_result['error']}")

## 🏆 Example Usage: Competitive Analysis

Analyze major players in AI-driven financial services:

In [ ]:
# Example 3: Competitive Analysis
print("🔍 Example 3: Competitive Analysis")
print("=" * 50)

competitive_result = app.analyze(
    industry="fintech",
    query="Who are the major players in AI-driven financial services and what are their strategies?",
    analysis_type="competitive"
)

# Display results
if 'error' not in competitive_result:
    print(f"\n📊 Analysis completed with {competitive_result['data_sources']} sources")
    print(f"🕐 Generated: {competitive_result['timestamp']}")
    print(f"🏢 Industry: {competitive_result['industry']}")
    print(f"❓ Query: {competitive_result['query']}")
    print(f"📈 Type: {competitive_result['analysis_type']}")
    print("\n💡 Key Insights:")
    print(competitive_result['insights'][:500] + "..." if len(competitive_result['insights']) > 500 else competitive_result['insights'])
else:
    print(f"❌ Error: {competitive_result['error']}")

## 📦 Example Usage: Batch Analysis

Process multiple queries simultaneously:

In [ ]:
# Example 4: Batch Analysis
print("🔍 Example 4: Batch Analysis")
print("=" * 50)

# Define batch queries
batch_queries = [
    {
        "industry": "automotive",
        "query": "Electric vehicle market trends and adoption rates",
        "analysis_type": "trends"
    },
    {
        "industry": "retail",
        "query": "Customer sentiment towards AI-powered shopping assistants",
        "analysis_type": "sentiment"
    },
    {
        "industry": "education",
        "query": "Major players in AI-driven educational technology",
        "analysis_type": "competitive"
    }
]

# Run batch analysis
batch_results = batch_analysis(app, batch_queries)

# Display summary
print("\n📊 BATCH ANALYSIS SUMMARY")
print("=" * 40)
for i, result in enumerate(batch_results, 1):
    if 'error' not in result:
        print(f"Analysis {i}: {result['industry']} - {result['analysis_type']} - {result['data_sources']} sources")
    else:
        print(f"Analysis {i}: Error - {result['error']}")

## 💾 Export Results

Save analysis results to files:

In [ ]:
# Export the latest analysis result
print("💾 Exporting Analysis Results")
print("=" * 40)

# Export the competitive analysis result
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
app.export_insights(competitive_result, f"latest_market_insights_{timestamp}.txt")

# Export analysis history to CSV
export_history_to_csv(app, f"market_analysis_history_{timestamp}.csv")

print("\n📊 Files exported successfully!")

## 📊 Create Analysis Dashboard

Generate visual dashboard of analysis results:

In [ ]:
# Create comprehensive dashboard
print("📊 Creating Analysis Dashboard")
print("=" * 40)

create_analysis_dashboard(app)

## 🎯 Interactive Analysis Interface

Run the interactive analysis tool (uncomment to use):

In [ ]:
# Uncomment the following line to run interactive analysis
# interactive_analysis(app)

print("🎯 Interactive Analysis Interface")
print("=" * 40)
print("To run interactive analysis, uncomment the line above and run this cell.")
print("\nThe interactive interface allows you to:")
print("1. Choose analysis type (trends, sentiment, competitive)")
print("2. Enter industry and query")
print("3. View results immediately")
print("4. Save results to files")
print("5. Process multiple queries in sequence")

## 📋 Analysis Summary

Review all completed analyses:

In [ ]:
# Display analysis summary
print("📋 Analysis Summary")
print("=" * 40)

history_df = app.get_analysis_history()

if not history_df.empty:
    print(f"Total analyses completed: {len(history_df)}")
    print(f"Industries analyzed: {', '.join(history_df['industry'].unique())}")
    print(f"Analysis types used: {', '.join(history_df['analysis_type'].unique())}")
    print(f"Average sources per analysis: {history_df['sources'].mean():.1f}")
    print(f"Total data sources processed: {history_df['sources'].sum()}")
    
    print("\n📊 Detailed History:")
    print(history_df.to_string(index=False))
else:
    print("No analyses completed yet. Run some analyses first!")

## 🎉 Conclusion

This Market Intelligence LLM application successfully combines:

### ✅ Key Features Implemented:
- **Real-time Data Retrieval**: Using Tavily AI for comprehensive market data
- **Advanced AI Analysis**: Leveraging OpenAI GPT-4o-mini for intelligent insights
- **Multiple Analysis Types**: Trends, sentiment, and competitive analysis
- **Batch Processing**: Efficient handling of multiple queries
- **Interactive Interface**: User-friendly query system
- **Visual Dashboards**: Comprehensive analysis visualization
- **Export Functionality**: Save results in multiple formats
- **Historical Tracking**: Complete analysis history management

### 🚀 Use Cases:
1. **Market Research**: Understand industry trends and opportunities
2. **Competitive Intelligence**: Analyze competitor strategies and market positioning
3. **Customer Insights**: Understand customer sentiment and preferences
4. **Strategic Planning**: Make data-driven business decisions
5. **Risk Assessment**: Identify potential market risks and challenges

### 🔧 Technical Stack:
- **Data Retrieval**: Tavily AI API
- **LLM Processing**: OpenAI GPT-4o-mini
- **Orchestration**: LangChain framework
- **Visualization**: Matplotlib and Seaborn
- **Data Processing**: Pandas
- **Environment**: Jupyter Notebook

### 🔮 Future Enhancements:
- Web interface using Streamlit
- Database integration for historical data
- Real-time monitoring and alerts
- Additional data sources integration
- Custom model fine-tuning
- API endpoint creation
- Cloud deployment options

---

**🎯 Ready to analyze market trends with AI!** Start by running your first analysis or explore the interactive interface above.

# Market Intelligence LLM Application

A comprehensive market intelligence application that combines **Tavily AI** for real-time data retrieval and **OpenAI GPT-4o-mini** for intelligent analysis and insight generation.

## 🚀 Features

- **Real-time Data Retrieval**: Fetch data from social media, news, and reports using Tavily AI
- **Three Analysis Types**: Market trends, customer sentiment, and competitive analysis
- **Batch Processing**: Analyze multiple queries simultaneously
- **Interactive Interface**: User-friendly query system
- **Comprehensive Dashboards**: Visual representation of analysis results
- **Export Functionality**: Save results in multiple formats (TXT, CSV)
- **Error Handling**: Robust error management and logging

## 📋 Prerequisites

Before running this notebook, ensure you have:
- Python 3.8 or higher
- API keys for Tavily AI and OpenAI
- Required packages installed (see requirements.txt)

In [ ]:
# Install required packages
!pip install langchain>=0.1.0 langchain-openai>=0.1.0 langchain-community>=0.0.20 tavily-python>=0.3.0 python-dotenv>=1.0.0 pandas>=1.5.0 matplotlib>=3.6.0 seaborn>=0.12.0 jupyter>=1.0.0

In [ ]:
# Import required libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dotenv import load_dotenv
import json
from typing import Dict, List, Any
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.schema import HumanMessage, SystemMessage

# Load environment variables
load_dotenv()

# Set up API keys
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Verify API keys are loaded
if not TAVILY_API_KEY:
    print("⚠️  TAVILY_API_KEY not found. Please set it in your .env file")
if not OPENAI_API_KEY:
    print("⚠️  OPENAI_API_KEY not found. Please set it in your .env file")

if TAVILY_API_KEY and OPENAI_API_KEY:
    print("✅ Environment setup complete!")
else:
    print("❌ Please configure your API keys before proceeding")

In [ ]:
class MarketIntelligenceApp:
    """
    A comprehensive market intelligence application that combines Tavily AI for real-time data retrieval
    and OpenAI GPT-4o-mini for intelligent analysis and insight generation.
    """
    
    def __init__(self):
        """Initialize the Market Intelligence Application with API clients."""
        self.tavily_tool = TavilySearchResults(max_results=10)
        self.llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.2,
            openai_api_key=OPENAI_API_KEY
        )
        self.analysis_history = []
        print("🚀 Market Intelligence App initialized successfully!")
    
    def analyze(self, industry: str, query: str, analysis_type: str) -> Dict[str, Any]:
        """
        Perform market intelligence analysis based on the specified type.
        
        Args:
            industry (str): The industry to analyze
            query (str): The specific query to analyze
            analysis_type (str): Type of analysis ('trends', 'sentiment', 'competitive')
            
        Returns:
            Dict[str, Any]: Analysis results with insights and metadata
        """
        try:
            # Step 1: Retrieve real-time data using Tavily
            print(f"🔍 Retrieving data for: {query}")
            search_results = self.tavily_tool.run(query)
            
            # Step 2: Prepare analysis prompt based on type
            analysis_prompts = {
                "trends": self._get_trends_prompt(),
                "sentiment": self._get_sentiment_prompt(),
                "competitive": self._get_competitive_prompt()
            }
            
            system_prompt = analysis_prompts.get(analysis_type, analysis_prompts["trends"])
            
            # Step 3: Analyze data using OpenAI GPT-4o-mini
            print(f"🧠 Analyzing data with {analysis_type} analysis...")
            
            messages = [
                SystemMessage(content=system_prompt),
                HumanMessage(content=f"""
                Industry: {industry}
                Query: {query}
                
                Data to analyze:
                {search_results}
                
                Please provide a comprehensive analysis based on this data.
                """)
            ]
            
            response = self.llm.invoke(messages)
            
            # Step 4: Structure the results
            result = {
                "timestamp": datetime.now().isoformat(),
                "industry": industry,
                "query": query,
                "analysis_type": analysis_type,
                "insights": response.content,
                "data_sources": len(search_results) if isinstance(search_results, list) else 10,
                "raw_data": search_results
            }
            
            # Step 5: Add to history
            self.analysis_history.append({
                "timestamp": result["timestamp"],
                "industry": industry,
                "analysis_type": analysis_type,
                "query": query,
                "sources": result["data_sources"]
            })
            
            print(f"✅ Analysis complete! Found {result['data_sources']} data sources")
            return result
            
        except Exception as e:
            print(f"❌ Error during analysis: {str(e)}")
            return {
                "error": str(e),
                "timestamp": datetime.now().isoformat(),
                "industry": industry,
                "query": query,
                "analysis_type": analysis_type
            }
    
    def _get_trends_prompt(self) -> str:
        """Get system prompt for market trends analysis."""
        return """You are a senior market analyst specializing in identifying and analyzing market trends. 
        Your task is to analyze the provided data and generate comprehensive insights about market trends.
        
        Focus on:
        1. Key market trends identified
        2. Growth opportunities
        3. Potential challenges
        4. Actionable recommendations for businesses
        
        Structure your response with clear headings and bullet points for easy readability."""
    
    def _get_sentiment_prompt(self) -> str:
        """Get system prompt for customer sentiment analysis."""
        return """You are a customer sentiment analysis expert. Your task is to analyze the provided data 
        and generate comprehensive insights about customer sentiment and opinions.
        
        Focus on:
        1. Overall sentiment distribution (positive, negative, neutral)
        2. Key themes in customer feedback
        3. Major concerns and pain points
        4. Opportunities for improvement
        
        Structure your response with clear headings and provide specific examples from the data."""
    
    def _get_competitive_prompt(self) -> str:
        """Get system prompt for competitive analysis."""
        return """You are a competitive intelligence analyst. Your task is to analyze the provided data 
        and generate comprehensive insights about the competitive landscape.
        
        Focus on:
        1. Key competitors and their market positions
        2. Competitive advantages and strategies
        3. Market share insights
        4. Innovation trends
        5. Strategic recommendations for businesses
        
        Structure your response with clear headings and provide actionable insights."""
    
    def get_analysis_history(self) -> pd.DataFrame:
        """Get analysis history as a pandas DataFrame."""
        return pd.DataFrame(self.analysis_history)
    
    def export_insights(self, result: Dict[str, Any], filename: str):
        """Export analysis insights to a text file."""
        try:
            report_content = f"""Market Intelligence Report
============================================================
Generated: {result['timestamp']}
Industry: {result['industry']}
Query: {result['query']}
Analysis Type: {result['analysis_type']}
Data Sources: {result['data_sources']}
============================================================

INSIGHTS:
{result['insights']}
"""
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(report_content)
            print(f"✅ Insights exported to {filename}")
        except Exception as e:
            print(f"❌ Error exporting insights: {str(e)}")
    
    def export_history_to_csv(self, filename: str):
        """Export analysis history to CSV file."""
        try:
            if self.analysis_history:
                df = self.get_analysis_history()
                df.to_csv(filename, index=False)
                print(f"✅ History exported to {filename}")
            else:
                print("⚠️  No analysis history to export")
        except Exception as e:
            print(f"❌ Error exporting history: {str(e)}")

print("📊 MarketIntelligenceApp class defined successfully!")

In [ ]:
def batch_analysis(app: MarketIntelligenceApp, queries: List[Dict[str, str]]) -> List[Dict[str, Any]]:
    """
    Perform batch analysis on multiple queries.
    
    Args:
        app (MarketIntelligenceApp): The application instance
        queries (List[Dict[str, str]]): List of query dictionaries with 'industry', 'query', and 'analysis_type'
        
    Returns:
        List[Dict[str, Any]]: List of analysis results
    """
    results = []
    total_queries = len(queries)
    
    print(f"🔄 Starting batch analysis of {total_queries} queries...")
    
    for i, query_data in enumerate(queries, 1):
        print(f"\n📊 Processing query {i}/{total_queries}")
        print(f"Industry: {query_data['industry']}")
        print(f"Query: {query_data['query']}")
        print(f"Analysis Type: {query_data['analysis_type']}")
        
        result = app.analyze(
            industry=query_data['industry'],
            query=query_data['query'],
            analysis_type=query_data['analysis_type']
        )
        
        results.append(result)
        
        # Add a small delay to avoid rate limiting
        import time
        time.sleep(1)
    
    print(f"\n✅ Batch analysis complete! Processed {len(results)} queries")
    return results

def interactive_analysis():
    """
    Interactive analysis interface for user-friendly query input.
    """
    print("🎯 Welcome to Interactive Market Intelligence Analysis!")
    print("=" * 60)
    
    app = MarketIntelligenceApp()
    
    while True:
        print("\n📋 Available Analysis Types:")
        print("1. trends - Market trends analysis")
        print("2. sentiment - Customer sentiment analysis")
        print("3. competitive - Competitive analysis")
        print("4. quit - Exit the application")
        
        choice = input("\nSelect analysis type (1-4): ").strip()
        
        if choice == '4' or choice.lower() == 'quit':
            print("👋 Thank you for using Market Intelligence Analysis!")
            break
        
        analysis_types = {'1': 'trends', '2': 'sentiment', '3': 'competitive'}
        
        if choice not in analysis_types:
            print("❌ Invalid choice. Please select 1-4.")
            continue
        
        analysis_type = analysis_types[choice]
        
        # Get user input
        industry = input("Enter industry: ").strip()
        query = input("Enter your query: ").strip()
        
        if not industry or not query:
            print("❌ Industry and query cannot be empty.")
            continue
        
        # Perform analysis
        result = app.analyze(industry, query, analysis_type)
        
        # Display results
        if 'error' not in result:
            print(f"\n📊 Analysis Results:")
            print(f"Industry: {result['industry']}")
            print(f"Query: {result['query']}")
            print(f"Analysis Type: {result['analysis_type']}")
            print(f"Data Sources: {result['data_sources']}")
            print(f"\n📈 Insights:")
            print(result['insights'])
            
            # Ask if user wants to export results
            export_choice = input("\nWould you like to export these results? (y/n): ").strip().lower()
            if export_choice == 'y':
                filename = input("Enter filename (without extension): ").strip()
                if filename:
                    app.export_insights(result, f"{filename}.txt")
        else:
            print(f"❌ Analysis failed: {result['error']}")
        
        # Continue or exit
        continue_choice = input("\nWould you like to perform another analysis? (y/n): ").strip().lower()
        if continue_choice != 'y':
            break
    
    return app

print("🛠️  Helper functions defined successfully!")

In [ ]:
def create_analysis_dashboard(app: MarketIntelligenceApp):
    """
    Create comprehensive dashboard visualizations for analysis results.
    
    Args:
        app (MarketIntelligenceApp): The application instance with analysis history
    """
    if not app.analysis_history:
        print("⚠️  No analysis history available for dashboard creation")
        return
    
    # Get analysis history
    df = app.get_analysis_history()
    
    # Set up the plotting style
    plt.style.use('seaborn-v0_8')
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Market Intelligence Analysis Dashboard', fontsize=16, fontweight='bold')
    
    # 1. Analysis Types Distribution
    analysis_counts = df['analysis_type'].value_counts()
    axes[0, 0].pie(analysis_counts.values, labels=analysis_counts.index, autopct='%1.1f%%', startangle=90)
    axes[0, 0].set_title('Analysis Types Distribution')
    
    # 2. Industries Analyzed
    if len(df['industry'].unique()) > 1:
        industry_counts = df['industry'].value_counts()
        axes[0, 1].bar(industry_counts.index, industry_counts.values, color='skyblue')
        axes[0, 1].set_title('Industries Analyzed')
        axes[0, 1].set_xlabel('Industry')
        axes[0, 1].set_ylabel('Number of Analyses')
        axes[0, 1].tick_params(axis='x', rotation=45)
    else:
        axes[0, 1].text(0.5, 0.5, 'Only one industry analyzed', ha='center', va='center', transform=axes[0, 1].transAxes)
        axes[0, 1].set_title('Industries Analyzed')
    
    # 3. Data Sources Distribution
    axes[1, 0].hist(df['sources'], bins=10, color='lightgreen', edgecolor='black')
    axes[1, 0].set_title('Data Sources Distribution')
    axes[1, 0].set_xlabel('Number of Sources')
    axes[1, 0].set_ylabel('Frequency')
    
    # 4. Analysis Timeline
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    timeline_data = df.groupby(df['timestamp'].dt.date).size()
    axes[1, 1].plot(timeline_data.index, timeline_data.values, marker='o', linewidth=2, markersize=6)
    axes[1, 1].set_title('Analysis Timeline')
    axes[1, 1].set_xlabel('Date')
    axes[1, 1].set_ylabel('Number of Analyses')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n📊 Analysis Summary Statistics:")
    print(f"Total Analyses: {len(df)}")
    print(f"Industries Covered: {len(df['industry'].unique())}")
    print(f"Analysis Types Used: {', '.join(df['analysis_type'].unique())}")
    print(f"Average Data Sources: {df['sources'].mean():.1f}")
    print(f"Date Range: {df['timestamp'].min().date()} to {df['timestamp'].max().date()}")

def display_analysis_results(result: Dict[str, Any]):
    """
    Display analysis results in a formatted and visually appealing way.
    
    Args:
        result (Dict[str, Any]): Analysis result dictionary
    """
    print("=" * 80)
    print("📊 MARKET INTELLIGENCE ANALYSIS RESULTS")
    print("=" * 80)
    
    if 'error' in result:
        print(f"❌ Analysis Error: {result['error']}")
        return
    
    # Basic information
    print(f"🕐 Generated: {result['timestamp']}")
    print(f"🏢 Industry: {result['industry']}")
    print(f"🔍 Query: {result['query']}")
    print(f"📈 Analysis Type: {result['analysis_type'].upper()}")
    print(f"📊 Data Sources: {result['data_sources']}")
    print("\n" + "=" * 80)
    print("💡 INSIGHTS:")
    print("=" * 80)
    print(result['insights'])
    print("\n" + "=" * 80)

def export_insights(result: Dict[str, Any], filename: str):
    """
    Export analysis insights to a text file with proper formatting.
    
    Args:
        result (Dict[str, Any]): Analysis result dictionary
        filename (str): Output filename
    """
    try:
        report_content = f"""Market Intelligence Report
============================================================
Generated: {result['timestamp']}
Industry: {result['industry']}
Query: {result['query']}
Analysis Type: {result['analysis_type']}
Data Sources: {result['data_sources']}
============================================================

INSIGHTS:
{result['insights']}
"""
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(report_content)
        print(f"✅ Insights exported to {filename}")
    except Exception as e:
        print(f"❌ Error exporting insights: {str(e)}")

def export_history_to_csv(app: MarketIntelligenceApp, filename: str):
    """
    Export analysis history to CSV file.
    
    Args:
        app (MarketIntelligenceApp): The application instance
        filename (str): Output filename
    """
    try:
        if app.analysis_history:
            df = app.get_analysis_history()
            df.to_csv(filename, index=False)
            print(f"✅ History exported to {filename}")
        else:
            print("⚠️  No analysis history to export")
    except Exception as e:
        print(f"❌ Error exporting history: {str(e)}")

print("📊 Visualization and export functions defined successfully!")

## 🚀 Initialize the Application

Now let's initialize the Market Intelligence Application and run some example analyses.

In [ ]:
# Initialize the Market Intelligence Application
app = MarketIntelligenceApp()

# Verify the application is ready
print("🎯 Market Intelligence Application is ready!")
print("Available methods:")
print("- app.analyze(industry, query, analysis_type)")
print("- app.get_analysis_history()")
print("- app.export_insights(result, filename)")
print("- app.export_history_to_csv(filename)")
print("\n📋 Analysis types: 'trends', 'sentiment', 'competitive'")
print("\n🔧 Helper functions:")
print("- batch_analysis(app, queries)")
print("- interactive_analysis()")
print("- create_analysis_dashboard(app)")
print("- display_analysis_results(result)")

## 📊 Example Usage

Let's run some example analyses to demonstrate the capabilities of our Market Intelligence Application.

In [ ]:
# Example 1: Market Trends Analysis
print("🔍 Example 1: Analyzing AI Technology Trends")
print("=" * 50)

trends_result = app.analyze(
    industry="technology",
    query="What are the latest trends in AI adoption and implementation?",
    analysis_type="trends"
)

# Display the results
display_analysis_results(trends_result)

# Export the results
export_insights(trends_result, "latest_market_insights.txt")

In [ ]:
# Example 2: Customer Sentiment Analysis
print("🔍 Example 2: Analyzing Customer Sentiment in Healthcare AI")
print("=" * 60)

sentiment_result = app.analyze(
    industry="healthcare",
    query="What is the customer sentiment towards AI-powered diagnostic tools?",
    analysis_type="sentiment"
)

# Display the results
display_analysis_results(sentiment_result)

In [ ]:
# Example 3: Competitive Analysis
print("🔍 Example 3: Competitive Analysis in Fintech")
print("=" * 50)

competitive_result = app.analyze(
    industry="fintech",
    query="Who are the major players in AI-driven financial services and what are their strategies?",
    analysis_type="competitive"
)

# Display the results
display_analysis_results(competitive_result)

In [ ]:
# Example 4: Batch Analysis
print("🔍 Example 4: Batch Analysis of Multiple Industries")
print("=" * 60)

# Define multiple queries for batch processing
batch_queries = [
    {
        "industry": "automotive",
        "query": "Electric vehicle market trends and adoption rates",
        "analysis_type": "trends"
    },
    {
        "industry": "retail",
        "query": "Customer sentiment towards AI-powered shopping assistants",
        "analysis_type": "sentiment"
    },
    {
        "industry": "artificial intelligence",
        "query": "Current trends in generative AI and large language models",
        "analysis_type": "trends"
    }
]

# Perform batch analysis
batch_results = batch_analysis(app, batch_queries)

# Display summary of batch results
print(f"\n📊 Batch Analysis Summary:")
print(f"Total queries processed: {len(batch_results)}")
for i, result in enumerate(batch_results, 1):
    if 'error' not in result:
        print(f"{i}. {result['industry']} - {result['analysis_type']} - {result['data_sources']} sources")
    else:
        print(f"{i}. Error: {result['error']}")

# Export the last result as an example
if batch_results and 'error' not in batch_results[-1]:
    export_insights(batch_results[-1], "system_test_results.txt")

In [ ]:
# Export analysis history to CSV
print("💾 Exporting Analysis History")
print("=" * 30)

export_history_to_csv(app, "market_analysis_history.csv")

# Display analysis history
print("\n📋 Analysis History:")
history_df = app.get_analysis_history()
print(history_df.to_string(index=False))

# Create analysis dashboard
print("\n📊 Creating Analysis Dashboard...")
create_analysis_dashboard(app)

## 🎯 Interactive Analysis

Run the interactive analysis tool to perform custom analyses. Uncomment and run the cell below to start the interactive interface.

In [ ]:
# Uncomment the line below to start interactive analysis
# interactive_app = interactive_analysis()

print("🎯 Interactive Analysis Ready!")
print("Uncomment the line above to start the interactive interface")
print("\nAlternatively, you can run individual analyses:")
print("result = app.analyze('industry', 'your query', 'analysis_type')")
print("display_analysis_results(result)")

## 🎉 Conclusion

Congratulations! You have successfully set up and run the Market Intelligence LLM Application. This comprehensive system provides:

### ✅ What You've Accomplished:

1. **Real-time Data Retrieval**: Integration with Tavily AI for current market data
2. **AI-Powered Analysis**: Leveraging OpenAI GPT-4o-mini for intelligent insights
3. **Multi-Type Analysis**: Market trends, customer sentiment, and competitive analysis
4. **Batch Processing**: Efficient analysis of multiple queries
5. **Visualization**: Dashboard and charts for data presentation
6. **Export Capabilities**: Save results in multiple formats

### 🚀 Next Steps:

1. **Customize Analysis**: Modify the analysis prompts to suit your specific needs
2. **Add More Data Sources**: Integrate additional APIs for broader data coverage
3. **Enhance Visualizations**: Create more detailed charts and graphs
4. **Build a Web Interface**: Consider using Streamlit or Flask for a web-based UI
5. **Implement Database Storage**: Store historical data for trend analysis
6. **Add Real-time Monitoring**: Set up alerts for significant market changes

### 🔧 Customization Tips:

- Modify the `_get_*_prompt()` methods to adjust analysis focus
- Add new analysis types by extending the `analyze()` method
- Customize visualization themes and styles
- Integrate with your existing business intelligence tools

### 📚 Resources:

- [Tavily AI Documentation](https://tavily.com/docs)
- [OpenAI API Documentation](https://platform.openai.com/docs)
- [LangChain Documentation](https://python.langchain.com/)
- [Pandas Documentation](https://pandas.pydata.org/docs/)

**Ready to analyze market trends with AI!** 🚀